In [1]:
import json
import sklearn
import requests
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from __future__ import division
import tensorflow as tf
import numpy as np
import tarfile
import os
import json
import matplotlib.pyplot as plt
import time

In [2]:
with open("./zenodo_open_metadata_06_04_2017.json", "r") as fp:
    data = json.load(fp)
spams = [17229, 400865, 19031, 285825, 440002, 292995, 221828, 18818, 160518, 32519, 437895, 22211, 290571, 438157, 44942, 33550, 246800, 51728, 398866, 400531, 60051, 50704, 32918, 164886, 164888, 495001, 165784, 34331, 321818, 60828, 346531, 399268, 18045, 27942, 35364, 30632, 202291, 47155, 293045, 47286, 290359, 224947, 29113, 375738, 20539, 17596, 293053, 439998, 439997, 290237, 49089, 440001, 440003, 440004, 440005, 440006, 440007, 154696, 440009, 440008, 290123, 35276, 284358, 398797, 32076, 33365, 249942, 290134, 344537, 166752, 291812, 18281, 46445, 51054, 290685, 34287, 28271, 439999, 232944, 290157, 23155, 290166, 440000, 249981]
maybe_spams = [153959, 12846, 13138, 13385, 398764, 400012]
for d in data:
    if d['recid'] in (spams + maybe_spams):
        d['spam'] = True
labels = [d['spam'] for d in data]

In [3]:
def feat_tr(d):
    return d['description'] + d['title']

X = [feat_tr(d) for d in data]
y = [d['spam'] for d in data]

In [4]:
ngram_range=(1, 1)
count_vect = CountVectorizer(ngram_range=ngram_range, max_features=5000)
tfid = TfidfTransformer()
X_v = count_vect.fit_transform(X)
X_v = tfid.fit_transform(X_v)
trainX, testX, trainY, testY = train_test_split(X_v, y, test_size=0.33, random_state=42)

In [7]:
trainX_v = np.asarray(trainX.todense())
trainY_v = np.array([float(y) for y in trainY])
testX_v = np.asarray(testX.todense())
testY_v = np.array([float(y) for y in testY])

In [68]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=trainX_v.shape[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearClassifier(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
#x = np.array([1., 2., 3., 4.])
#y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":trainX_v}, trainY_v, batch_size=1000,
                                              num_epochs=10)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
#print(estimator.evaluate(input_fn=input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_num_ps_replicas': 0, '_environment': 'local', '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f294395bf98>, '_num_worker_replicas': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_model_dir': '/tmp/tmpf0s_0rem', '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_master': '', '_task_type': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_is_chief': True, '_session_config': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x7f2943ef7978>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=5000, default_value=None, dtype=tf.float32, normalizer=None)], 'joint_weights': False, 'optimizer': None, 'gradient_clip_norm': None})

In [60]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":trainX_v}, trainY_v, batch_size=500,
                                              num_epochs=1000)

In [62]:
estimator.fit(input_fn=input_fn, steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpfu2i5chz/model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/tmpfu2i5chz/model.ckpt.
INFO:tensorflow:step = 2001, loss = 0.0651012
INFO:tensorflow:global_step/sec: 44.109
INFO:tensorflow:step = 2101, loss = 0.0665069 (2.268 sec)
INFO:tensorflow:global_step/sec: 45.1782
INFO:tensorflow:step = 2201, loss = 0.0592572 (2.213 sec)
INFO:tensorflow:global_step/sec: 45.169
INFO:tensorflow:step = 2301, loss = 0.0522081 (2.214 sec)
INFO:tensorflow:global_step/sec: 45.84
INFO:tensorflow:step = 2401, loss = 0.0323073 (2.181 sec)
INFO:tensorflow:global_step/sec: 45.

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x7f29429555f8>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=5000, default_value=None, dtype=tf.float32, normalizer=None)], 'joint_weights': False, 'optimizer': None, 'gradient_clip_norm': None})

In [63]:
input_fn_test = tf.contrib.learn.io.numpy_input_fn({"x":testX_v}, batch_size=testX_v.shape[0],
                                              num_epochs=1)

In [64]:
y_out = estimator.predict(input_fn=input_fn_test)
y_out = np.array(list(y_out))
y_out2 = list(y_out > 0.5)
acc = [(ref, pred) for ref, pred in zip(testY, y_out2)]
c = Counter(acc)
print(c)
print("Spam->Spam: {0:.4f}".format(c[(True, True)] / (c[(True, True)] + c[(True, False)])))
print("Ham -> Ham: {0:.4f}".format(c[(False, False)] / (c[(False, False)] + c[(False, True)])))

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/tmpfu2i5chz/model.ckpt-3000
Counter({(False, False): 57653, (True, False): 1846, (False, True): 1192, (True, True): 29})
Spam->Spam: 0.0155
Ham -> Ham: 0.9797


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
#print(estimator.evaluate(input_fn=input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_model_dir': '/tmp/tmpnbcll3pl', '_save_checkpoints_secs': 600, '_environment': 'local', '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f70be88fdd8>, '_master': '', '_task_type': None, '_task_id': 0, '_is_chief': True, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_save_summary_steps': 100}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create

LinearRegressor(params={'optimizer': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7f703c6430f0>, 'gradient_clip_norm': None, 'joint_weights': False, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)]})

In [82]:
x2 = np.array([2., 3., 5., 4.])

In [113]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x2}, batch_size=4,
                                              num_epochs=1)

In [114]:
y2 = list(estimator.predict(input_fn=input_fn))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/tmpnbcll3pl/model.ckpt-1000


In [115]:
len(y2)

4

In [116]:
y2

[-1.000056, -3.9998956, -2.999949, -2.0000024]